In [74]:
import io
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import FreqDist
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud 
import nltk
from nltk import FreqDist

In [37]:
with open("book.txt", "r", encoding='utf8') as f:
    file = f.read()

In [38]:
raw = file.split("Chapter")[1:]
print(len(raw))

61


In [43]:
book_df = pd.DataFrame({"chapter": raw})

In [51]:
chapters = book_df.copy()
chapters['chapter_num'] = chapters.index+1

In [53]:
chapters

,chapter,chapter_num
0,1\n\n It is a truth universally acknowle...,1
1,2\n\n Mr. Bennet was among the earliest ...,2
2,"3\n\n Not all that Mrs. Bennet, however,...",3
3,4\n\n When Jane and Elizabeth were alone...,4
4,5\n\n Within a short walk of Longbourn l...,5
...,...,...
56,57\n\n The discomposure of spirits which...,57
57,58\n\n Instead of receiving any such let...,58
58,"59\n\n “My dear Lizzy, where can you hav...",59
59,60\n\n Elizabeth’s spirits soon rising t...,60


In [55]:
chapters['chapter'][0]

' 1\n\n      It is a truth universally acknowledged, that a single man in\n      possession of a good fortune, must be in want of a wife.\n\n      However little known the feelings or views of such a man may be\n      on his first entering a neighbourhood, this truth is so well\n      fixed in the minds of the surrounding families, that he is\n      considered the rightful property of some one or other of their\n      daughters.\n\n      “My dear Mr. Bennet,” said his lady to him one day, “have you\n      heard that Netherfield Park is let at last?”\n\n      Mr. Bennet replied that he had not.\n\n      “But it is,” returned she; “for Mrs. Long has just been here, and\n      she told me all about it.”\n\n      Mr. Bennet made no answer.\n\n      “Do you not want to know who has taken it?” cried his wife\n      impatiently.\n\n      “_You_ want to tell me, and I have no objection to hearing it.”\n\n      This was invitation enough.\n\n      “Why, my dear, you must know, Mrs. Long says th

In [56]:
def unnest_tokens(dataframe, column, tokenizer, new_column='word'): 
    """
    Applies a tokenizer to a column and then unnest the dataframe
    
    dataframe: dataframe with text
    column: name of the column with the text
    new_column: what you want the column of words to be called
    tokenizer: a function to be used to tokenize the text
    """ 
    new_column_num = new_column + "_num"
    df = ( dataframe[column]
              .apply(tokenizer)
              .apply(pd.Series)
              .stack()
              .reset_index(level=1)
              .rename(columns={0: new_column, "level_1":new_column_num})
              .join(dataframe.drop(column, axis=1), how='left')
              .reset_index(drop=True)
         )
    df[new_column_num] = pd.to_numeric(df[new_column_num])
    df[new_column_num] = df[new_column_num] + 1
    return df

In [62]:
chapters['chapter'] = chapters.chapter.str.replace("\n", " ")
chapters.chapter[0]

' 1        It is a truth universally acknowledged, that a single man in       possession of a good fortune, must be in want of a wife.        However little known the feelings or views of such a man may be       on his first entering a neighbourhood, this truth is so well       fixed in the minds of the surrounding families, that he is       considered the rightful property of some one or other of their       daughters.        “My dear Mr. Bennet,” said his lady to him one day, “have you       heard that Netherfield Park is let at last?”        Mr. Bennet replied that he had not.        “But it is,” returned she; “for Mrs. Long has just been here, and       she told me all about it.”        Mr. Bennet made no answer.        “Do you not want to know who has taken it?” cried his wife       impatiently.        “_You_ want to tell me, and I have no objection to hearing it.”        This was invitation enough.        “Why, my dear, you must know, Mrs. Long says that Netherfield is       take

## Get Sentences

In [64]:
import spacy
nlp = spacy.load('en')

In [65]:
def sentence_tokenizer(text):
    doc = nlp(text)
    sentences = [sent.string.strip() for sent in doc.sents]
    return sentences

In [66]:
sentences = unnest_tokens(chapters, "chapter", sentence_tokenizer, "sentence")

In [68]:
sentences

,sentence_num,sentence,chapter_num
0,1,1,1
1,2,"It is a truth universally acknowledged, that a...",1
2,3,However little known the feelings or views of ...,1
3,4,"“My dear Mr. Bennet,” said his lady to him one...",1
4,5,Mr. Bennet replied that he had not.,1
...,...,...,...
6975,47,"Elizabeth’s instructions, she began to compreh...",61
6976,48,Lady Catherine was extremely indignant on the ...,61
6977,49,"But at length, by Elizabeth’s persuasion...",61
6978,50,"With the Gardiners, they were always on the mo...",61


## Get Words

In [69]:
def word_tokenizer(text):
    doc = nlp(text)
    words = [[token.text, token.lemma_, token.pos_, token.is_stop] for token in doc]
    return words

In [70]:
words = unnest_tokens(sentences, "sentence", word_tokenizer, "word")

In [71]:
words.head()

,word_num,word,sentence_num,chapter_num
0,1,"[1, 1, X, False]",1,1
1,1,"[It, -PRON-, PRON, True]",2,1
2,2,"[is, be, AUX, True]",2,1
3,3,"[a, a, DET, True]",2,1
4,4,"[truth, truth, NOUN, False]",2,1


In [72]:
words[['word', 'lemma', 'POS', "stop"]] = pd.DataFrame(words.word.values.tolist())

In [73]:
words.head()

,word_num,word,sentence_num,chapter_num,lemma,POS,stop
0,1,1,1,1,1,X,False
1,1,It,2,1,-PRON-,PRON,True
2,2,is,2,1,be,AUX,True
3,3,a,2,1,a,DET,True
4,4,truth,2,1,truth,NOUN,False


In [75]:
words.to_csv("words.csv")